# Kenzie's DeepLabCut Workflow with Jupyter

The goal of this is notebook is to bring together the general guidance on using DeepLab cut, while making the analysis executable in context with the direction.  The first step will be to activate the relevant `conda` environment, which contains DeepLabCut. In the case of the GPU computer, this will be done by launching the *anaconda prompt (anaconda powershell prompt is also fine)* in administator mode and typing `conda activate dlc-windowsGPU-2023'`  While there are other 'conda' environments, this is the most up to date.

## General Overview

1. Import relevant packages and create project
2. Extract frames from imported videos
3. Label frames from videos to denote anatomincal landmarks
4. ...

In [1]:
# %% Imports
import os
import platform
from datetime import datetime
import deeplabcut

2023-10-26 02:52:13.846035: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-26 02:52:15.882576: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-26 02:52:15.882625: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-10-26 02:52:16.123814: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-26 02:52:27.994407: W tensorflow/stream_executor/platform/de

Loading DLC 2.3.6...


/home/kenzie_mackinnon/mambaforge/envs/dlc/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# All the functions that I will be using

# Lists all files in a directory
def filePathList(dir_path):
    file_paths = []
    for root, dirs, files in os.walk(dir_path):
        for file in files:
            file_paths.append(os.path.join(root, file))
    return file_paths

# Converts file paths in between Unix and DOS
def pathconvUnixToDos(paths):
    dos_paths = []
    for path in paths:
        dos_path = path.replace('/', '\\')
        dos_path = "C:" + dos_path
        dos_paths.append(dos_path)
    return dos_paths

# Checks to see if a directory is already present
def directoryPresent(targetPath, projName):
    path = os.path.join(projName, targetPath)

    # Will return boolean
    return os.path.isdir(path)

Now all we have to do is give two paths, which are where we want to put the project `targetForProject` and where are the videos we want to import currently stored `videoImportPath`

In [3]:
# Gathering User input
projectName = str(input("Enter a project name: "))
experimenterName = str(input("Enter name of experimenter: "))
targetForProject = str(input("Give path of parent folder you would like to place project in: "))
videoImportPath = str(input("Enter file path for videos you want to import: "))

This will take the paths given and put together the command that has to be done to create a new project.  It will also account for the operating system used at the time and adjust the list of video file paths accordingly making this work on whatever system is being used.  Furthermore, to prevent overwriting a project that might already exist in the place where you want to put the project, there is also a check to make sure that the new project is the same.

In [ ]:
# Depending on the OS processing the file we will run different functions to create file path lists
operatingSystem = platform.system()

if operatingSystem == "Windows":
    print("Window operating system detected!")

    # Path where recorded videos can currently be found
    videoImportPath = str(input("Enter file path for videos you want to import: "))

    # Creating list of file paths for each video in specified folder
    file_paths = filePathList(videoImportPath)

    # Changing file paths from Unix format to DOS for videos
    dos_path_conversion = pathconvUnixToDos(file_paths)
    file_paths = dos_path_conversion

    # Changing file paths from Unix format to DOS for target directory
    # unixPathComponents = targetForProject.split('/')
    # targetForProject = os.path.join(*unixPathComponents).replace('/', os.sep)
    # print(targetForProject)
elif operatingSystem == "Linux":
    print("Linux operating system detected!")
    # Path where recorded videos can currently be found
    videoImportPath = str(input("Enter file path for videos you want to import: "))

    # Creating list of file paths for each video in specified folder
    file_paths = filePathList(videoImportPath)
elif operatingSystem == "Darwin":
    print("Darwin(MacOS) operating system detected!")
    # Path where recorded videos can currently be found
    videoImportPath = str(input("Enter file path for videos you want to import: "))

    # Creating list of file paths for each video in specified folder
    file_paths = filePathList(videoImportPath)
else:
    print("Operating system not detected!")
    print("Falling back onto Unix path protocol")
    # Path where recorded videos can currently be found
    videoImportPath = str(input("Enter file path for videos you want to import: "))

    # Creating list of file paths for each video in specified folder
    file_paths = filePathList(videoImportPath)

# Checking result of variable file path importing
print("Project Name: " + projectName)
print("Experimenter: " + experimenterName)
print("Output of file paths:")
print("--------------------")
print(file_paths)
# %% Creation of project

# Checking to see if project with same name alredy exists
current_date = datetime.now().strftime("%Y-%m-%d")

newProjectName = projectName + "-" + experimenterName + "-" + current_date

if directoryPresent(newProjectName, targetForProject):
    print(f"Directory {newProjectName} already exists in {targetForProject}.")
else:
    print(f"Directory {newProjectName} does not exist in {targetForProject}.")
    config_path = deeplabcut.create_new_project(projectName, experimenterName, file_paths, working_directory=(targetForProject), copy_videos=True)

**TODO**: Before running next command, make sure `config.yaml` has the appropriate information.

In [ ]:
# %% Extract frames from videos
deeplabcut.extract_frames(config_path, mode='automatic', userfeedback=False)

In [ ]:
# %% Label frames
deeplabcut.label_frames(config_path)

In [ ]:
# %% Check Annotated Frames
deeplabcut.check_labels(config_path, visualizeindividuals=True)


## Start training of Feature Detectors

This function trains the network for a specific shuffle of the training dataset. The user can set various parameters in the project `config.yaml`

Training can be stopped at any time. Note that the weights are only stored every 'save_iters' steps. For this demo the state it is advisable to store & display the progress very often. In practice this is inefficient.

In [ ]:
deeplabcut.train_network(path_config_file, shuffle=1, displayiters=10, saveiters=100)

**Note, that if it reaches the end or you stop it (by hitting "stop" or by CTRL+C), you will see an "KeyboardInterrupt" error, but you can ignore this!**

## Evaluate a trained network

This function evaluates a trained model for a specific shuffle/shuffles at a particular training state (snapshot) or on all the states. The network is evaluated on the data set (images) and stores the results as .csv file in a subdirectory under **evaluation-results**.

You can change various parameters in the ```config.yaml``` file of this project. For evaluation all the model descriptors (Task, TrainingFraction, Date etc.) are important. For the evaluation one can change pcutoff. This cutoff also influences how likely estimated positions need to be so that they are shown in the plots. One can furthermore, change the colormap and dotsize for those graphs.

In [ ]:
deeplabcut.evaluate_network(path_config_file,plotting=False)

*NOTE: depending on your set up sometimes you get some "matplotlib errors, but these are not important*

Now you can go check out the images. Given the limited data input and it took ~20 mins to test this out, it is not meant to track well, so don't be alarmed. This is just to get you familiar with the workflow... 

## Analyzing videos
This function extracts the pose based on a trained network from videos. The user can choose the trained network - by default the most recent snapshot is used to analyse the videos. However, the user can also specify the snapshot index for the variable **snapshotindex** in the **config.yaml** file).

The results are stored in hd5 file in the same directory, where the video resides. The pose array (pose vs. frame index) can also be exported as csv file (set flag to...). 

In [ ]:
# Creating video path:
videofile_path = ('/Users/kenzie_mackinnon/deeplabcut/projects/mac-perturbation_treadmill-kenzie-2023-02-08/videos/m3v1mp4.mp4')

In [ ]:
print("Start analyzing the video!")
#our demo video on a CPU with take ~30 min to analze! GPU is much faster!
deeplabcut.analyze_videos(path_config_file,[videofile_path])

## Create labeled video

This function is for the visualization purpose and can be used to create a video in .mp4 format with the predicted labels. This video is saved in the same directory, where the (unlabeled) video resides. 

Various parameters can be set with regard to the colormap and the dotsize. The parameters of the 

In [ ]:
deeplabcut.create_labeled_video(path_config_file,[videofile_path])

## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color. The underlying functions can easily be customized.

In [ ]:
%matplotlib notebook
deeplabcut.plot_trajectories(path_config_file,[videofile_path],showfigures=True)

#These plots can are interactive and can be customized (see https://matplotlib.org/)

## Extract outlier frames, where the predictions are off.

This is optional step allows to add more training data when the evaluation results are poor. In such a case, the user can use the following function to extract frames where the labels are incorrectly predicted. Make sure to provide the correct value of the "iterations" as it will be used to create the unique directory where the extracted frames will be saved.

In [ ]:
deeplabcut.extract_outlier_frames(path_config_file,[videofile_path])

The user can run this iteratively, and (even) extract additional frames from the same video.

## Manually correct labels

This step allows the user to correct the labels in the extracted frames. Navigate to the folder corresponding to the video 'm3v1mp4' and use the GUI as described in the protocol to update the labels.

In [ ]:
%gui wx
deeplabcut.refine_labels(path_config_file)

In [ ]:
#Perhaps plot the labels to see how how all the frames are annotated (including the refined ones)
deeplabcut.check_labels(path_config_file)

In [ ]:
# Now merge datasets (once you refined all frames)
deeplabcut.merge_datasets(path_config_file)

## Create a new iteration of training dataset, check it and train...

Following the refine labels, append these frames to the original dataset to create a new iteration of training dataset.

In [ ]:
deeplabcut.create_training_dataset(path_config_file)

Now one can train the network again... (with the expanded data set)

In [ ]:
deeplabcut.train_network(path_config_file, shuffle=1)